# Hello

### Importar librerias

In [1]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import classification_report,accuracy_score
import json
import pickle

c:\Users\Ruben\OneDrive\Documentos\ITESM_IDM\Semestre 8\BloqueIngenieria\semana2\cloudComputing_act\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Extraer datos

In [2]:
#path = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")
#data = pd.read_csv(path+"/data.csv")

In [3]:
data = pd.read_csv("../data/pca_70_componentes.csv")

In [4]:
data.head(10)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,Bankrupt?
0,-7.338294,0.373294,-0.309014,-1.021642,0.162798,1.448300,-0.710591,1.195317,-0.535536,1.665768,...,-0.263133,0.400478,0.172639,0.049200,0.039152,0.121028,-0.294886,-0.240876,0.301496,1
1,-2.703713,-0.986346,-2.155617,-2.473644,0.835692,0.607047,-0.324440,1.361657,-0.409246,0.693192,...,0.107572,0.005134,0.104500,0.232198,-0.122784,0.177915,0.068680,0.023459,0.161399,1
2,-4.307059,-0.404700,-0.309801,-0.729865,0.335723,-0.198898,0.016022,0.562789,-0.501530,1.793298,...,-0.401372,0.294859,0.120255,-0.488918,-0.060014,0.173675,-0.225839,0.233203,0.496183,1
3,-5.830736,-1.385669,-1.301105,-2.160071,0.731532,1.043664,0.558476,-0.195704,0.705732,0.024977,...,0.278200,0.105868,0.190450,0.320940,0.154974,0.209494,0.516877,-0.046809,-0.021523,1
4,-2.343197,-0.201748,0.806267,-0.894912,0.203652,0.266931,0.349238,-0.201997,0.604713,-1.176532,...,-0.055855,-0.139035,0.213745,-0.016422,-0.096588,0.003419,0.020940,-0.105379,0.332278,1
5,-6.681856,-0.774171,-0.927413,-1.612984,0.432967,2.471917,0.463235,0.190965,0.567973,1.592355,...,-0.286507,0.386659,-0.308795,0.081220,0.241446,0.003008,0.378032,0.006843,-0.105714,1
6,-5.855967,0.082141,-0.627971,-0.033042,0.187159,1.688968,-1.135885,2.090697,-0.923283,0.750207,...,-0.039860,0.016310,0.286333,-0.271483,-0.231726,0.346901,-0.378374,0.156045,0.444376,0
7,-0.448640,0.203617,0.487619,-1.186015,0.333833,0.580163,0.741092,-0.258178,0.830105,-1.716014,...,0.047655,0.156581,0.035558,0.151944,-0.093844,0.011249,0.039839,-0.053214,0.130370,0
8,-0.530649,-1.312022,-0.004166,0.584962,0.009090,-0.724902,0.192013,-0.566441,0.488995,-0.669149,...,0.051884,-0.213961,0.320394,0.310649,-0.256178,-0.061907,-0.055822,-0.107143,-0.050371,0
9,-0.366075,0.735611,1.246965,-0.469174,0.144420,1.100224,0.966121,-0.390257,1.050538,-1.697269,...,-0.132694,0.225423,-0.464709,-0.095799,-0.245637,-0.085799,0.373590,-0.251319,0.086150,0


In [5]:
data.shape

(6819, 71)

### Modelo

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["Bankrupt?"], axis=1), data["Bankrupt?"], test_size=0.2, random_state=42)
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70
3759,-1.140608,0.480156,-0.759926,-2.796154,0.772126,0.727323,1.014923,-0.145113,0.590672,-0.727444,...,0.176373,0.317569,0.097250,-0.195421,-0.298902,-0.025103,0.086656,-0.108761,0.168851,-0.058053
1782,-0.260267,0.572487,2.348049,2.301804,-0.597558,-0.080781,-0.040810,-0.003390,-0.015728,0.893517,...,0.034468,-0.071443,0.196690,-0.038839,0.183687,-0.032770,-0.149872,-0.315393,-0.054966,-0.155090
5013,0.900051,-0.714695,-1.732955,-0.897534,0.216791,-0.380074,-0.364174,0.109163,-0.571839,1.182433,...,-0.056607,-0.165086,-0.147754,0.126504,-0.238337,-0.395436,-0.043125,0.070954,0.031566,0.042335
5412,-1.186987,3.418958,3.690440,-0.698332,-0.072059,0.024235,0.404884,0.268749,0.296243,-1.899067,...,0.238380,-0.095425,0.216301,0.216759,-0.038959,-0.418429,0.098801,0.103038,-0.076093,-0.086596
3066,-1.221979,0.844584,1.145609,-0.769437,0.152215,0.142710,0.347644,-0.029464,0.163310,-0.357528,...,0.276906,0.172637,0.288408,0.170001,0.432684,-0.385143,0.043239,0.018138,-0.054362,0.087772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,5.085515,1.215414,1.088639,-0.240453,-0.176133,-1.321257,1.384407,-2.406528,0.726128,0.002931,...,-0.327602,-0.070451,0.228569,-0.400663,-0.311196,0.165020,-0.182144,0.157920,-0.234649,0.014043
5191,3.875070,0.616553,0.584459,0.515760,-0.168247,-0.524600,0.098302,-0.687327,-0.022449,-0.026363,...,-0.027196,-0.338576,0.240504,-0.308415,-0.150755,-0.047118,-0.186410,-0.096629,-0.031947,0.080931
5226,-0.117474,0.541386,1.877499,1.377089,-0.322174,0.679777,0.142341,0.085510,0.714094,-0.837932,...,-0.229690,0.077015,-0.298922,0.175991,-0.038856,0.085346,-0.012557,0.195537,0.017068,-0.013132
5390,4.052295,0.495118,0.217813,0.561029,-0.272347,-0.225973,-0.274712,-0.067791,-0.161443,-0.312956,...,-0.119632,0.205514,-0.065520,0.002991,0.253221,0.106078,-0.064710,-0.111399,0.047141,-0.026120


In [7]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [8]:
def sigmoid(x):
    return [1 / (1 + np.exp(-y)) for y in x]

y_pred = sigmoid(y_pred)

y_pred_sort = {i:y_pred[i] for i in range(len(y_pred))}
y_pred_sort = dict(sorted(y_pred_sort.items(), key=lambda x: x[1], reverse=True))

how_many_ones = y_test.value_counts()[1]

one_predictions = list(y_pred_sort.keys())[:how_many_ones]

final_preds = [1 if x in one_predictions else 0 for x in range(len(y_pred))]

In [9]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1313
           1       0.39      0.39      0.39        51

    accuracy                           0.95      1364
   macro avg       0.68      0.68      0.68      1364
weighted avg       0.95      0.95      0.95      1364



In [10]:
X_test

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70
239,-2.508763,-3.236584,-2.634913,-0.471904,0.545137,-0.372686,0.579150,-0.970863,0.184079,0.388427,...,-0.366115,-0.139270,-0.173363,-0.058660,0.299354,-0.009820,-0.020676,0.011149,-0.001770,-0.103003
2850,0.554196,0.948523,0.697611,-1.196512,0.254458,0.489612,0.528982,-0.157908,0.467491,-1.310643,...,0.077308,0.151265,0.146626,-0.097572,-0.144836,-0.142659,-0.021144,0.054780,0.056249,-0.253083
2687,3.204800,0.483832,0.528284,0.516076,-0.131221,1.569525,0.561481,-0.110738,0.914215,-1.201075,...,0.014333,-0.376596,-0.295510,-0.144480,0.253215,-0.190163,-0.150254,0.221584,-0.283830,0.105303
6500,-5.271985,2.967743,2.454384,1.774013,-0.483117,-0.015024,-0.965084,1.180668,-0.749741,0.226518,...,-0.145560,0.521017,0.191564,0.009810,0.072119,-0.183349,0.172124,-0.179635,0.088720,-0.039537
2684,-1.350313,0.180171,-2.083013,-2.409058,0.722842,0.991949,-0.215720,1.092622,-0.366141,0.814691,...,0.553283,0.112413,-0.150912,0.021125,0.079041,-0.089414,0.099959,-0.020228,-0.196904,0.145683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,0.514038,1.313538,1.760316,-1.034971,0.073677,0.288101,0.818204,-0.738057,0.717431,-1.290878,...,0.190106,0.013700,-0.006912,-0.281742,0.171810,-0.408579,-0.149356,0.482562,-0.578285,0.050557
3946,0.981786,-1.853898,0.566422,4.260684,-0.805452,-1.585961,-0.686697,-0.611576,0.042463,-0.635425,...,-0.750465,0.302672,-0.472108,0.122332,-0.208650,0.500311,0.004848,0.260659,-0.045809,0.051568
5491,-2.099497,0.530659,-0.590708,-1.418302,0.358275,1.015674,0.036807,0.803366,-0.110907,0.524821,...,0.535561,0.361764,0.034026,0.150004,0.100991,-0.333576,0.155550,-0.005527,-0.037501,0.014014
2112,-3.671366,1.539616,1.930818,-0.678460,0.141061,-1.495039,-0.246367,0.226574,-0.701971,-0.100105,...,-0.466060,-0.426162,-0.180644,-1.150982,-1.330372,0.652530,-0.332215,-0.282182,0.383719,-0.078111


In [11]:
accuracy = accuracy_score(y_test,final_preds)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9545454545454546


In [12]:
umbral = min([y_pred_sort[x] for x in one_predictions])

print(f"CÓMO PREDECIR CON ESTE MODELO:\n\n Paso 1.- Calcularlo con la regresión lineal.\n PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].\n PASO 3.- Si el resultado es mayor a {umbral} entonces predecir 1.\n          De lo contrario, predecir 0.\n\nAsegúrense de meter esto en score.py")


umb = json.dumps({"umbral": [umbral]})
file = open("umbral.json", "w")
file.write(umb)
file.close()

file = open('model.pkl', 'wb')
pickle.dump(reg, file)
file.close()


CÓMO PREDECIR CON ESTE MODELO:

 Paso 1.- Calcularlo con la regresión lineal.
 PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].
 PASO 3.- Si el resultado es mayor a 0.5444075450750484 entonces predecir 1.
          De lo contrario, predecir 0.

Asegúrense de meter esto en score.py
